In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
from pyspark.streaming import StreamingContext

In [4]:
# Time interval in seconds
HOUR = 60*60
STANDARD = 10

ssc = StreamingContext(sc, STANDARD)

In [5]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.pprint()

In [6]:
# Change the path below to match yours
lines.repartition(1).saveAsTextFiles("file:///C:/Users/wille/spark/MyData/")

In [7]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

In [ ]:
# Don't run this cell unless you want to stop
ssc_t.stop()

In [ ]:
# Remove all empty files (run this code every so often).

import os
import shutil

data_folder = "C:/Users/wille/spark/MyData/"
mined_dirs = os.listdir(data_folder)
for dir in mined_dirs:
    files = os.listdir(data_folder + dir)
    if 'part-00000' in files:
        mined_data_pathname = data_folder + dir + '/part-00000'
        file = open(mined_data_pathname, "r")
        content = file.read()
        file.close()
        if len(content) == 0:
            for filename in os.listdir(data_folder + dir):
                try:
                    os.remove(data_folder + dir + '/' + filename)
                except:
                    print("an exception occured")
    if len(os.listdir(data_folder + dir)) == 0:
        os.rmdir(data_folder + dir)